In [1]:
import math

In [30]:
class Lattice:
    def printLattice(self):
        for t, level in enumerate(self.lattice):
            print 'level {0}'.format(t)
            level = [ round(elem, 30) for elem in level ]
            print ', '.join(map(str, level))

In [3]:
class RateLattice(Lattice):
    def __init__(self, n, S0, u, d):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                rate = S0 * u**j * d**(i - j)
                level.append(rate)
            self.lattice.append(level)

In [4]:
class ZCBLattice(Lattice):
    def __init__(self, F, q, n, rateLattice):
        self.lattice = []
        print "Calculating prices of zcb"
        clippedRate = rateLattice[:n+1]
        rightLevel = []
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    newLevel.append(F)
            else:
                for j in range(len(level)):
                    discount = 1.+clippedRate[n-i][j]/100.
                    price = (q*rightLevel[j+1]+(1-q)*rightLevel[j])/discount
                    newLevel.append(price)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [5]:
class CBLattice(Lattice):
    def __init__(self, F, q, n, c, rateLattice):
        self.lattice = []
        print "Calculating prices of cb"
        coupon = F*c
        clippedRate = rateLattice[:n+1]
        rightLevel = [F+coupon]*(n+1)
        self.lattice.insert(0, rightLevel)
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            for j in range(len(level)):
                spotRate = clippedRate[n-i-1][j]/100.
                price = coupon+(q*rightLevel[j+1]+(1-q)*rightLevel[j])/(1+spotRate)
                newLevel.append(price)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [6]:
class OptionLattice(Lattice):
    def __init__(self, n, q, K, isCall, isAmerican, rateLattice, baseLattice):
        multiplier = 1 if isCall else -1
        clippedBase = baseLattice[:n+1]
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print "Calculating options"
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    newLevel.append(max(multiplier * (level[j]-K), 0))
            else:
                for j in range(len(level)):
                    earlyExercise = max(multiplier * (level[j]-K), 0)
                    discount = 1.+clippedRate[n-i][j]/100.
                    hold = (q*rightLevel[j+1] + (1-q)*rightLevel[j])/discount
                    if earlyExercise > hold and isAmerican:
                        print "At time {0}, it's better to early exercise {1} than hold {2}".format(n-i, earlyExercise, hold)
                    newPrice = max(hold, earlyExercise) if isAmerican else hold
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [7]:
class ForwardLattice(Lattice):
    def __init__(self, F, q, n, c, rateLattice, baseLattice):
        self.lattice = []
        print "Calculating forwards of bond lattice"
        coupon = F*c
        clippedBase = baseLattice[:n+1]
        clippedRate = rateLattice[:n+1]
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    bondPrice = clippedBase[n-i][j]
                    price = bondPrice - coupon
                    newLevel.append(price)
            else:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i][j]/100.
                    price = (q*rightLevel[j+1]+(1-q)*rightLevel[j])/(1+spotRate)
                    newLevel.append(price)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [8]:
class FutureLattice(Lattice):
    def __init__(self, F, q, n, c, baseLattice):
        self.lattice = []
        print "Calculating futures of bond lattice"
        coupon = F*c
        clippedBase = baseLattice[:n+1]
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    bondPrice = clippedBase[n-i][j]
                    price = bondPrice - coupon
                    newLevel.append(price)
            else:
                for j in range(len(level)):
                    price = q*rightLevel[j+1]+(1-q)*rightLevel[j]
                    newLevel.append(price)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [9]:
class SwapLattice(Lattice):
    def __init__(self, q, n, rf, firstPaymentTime, payFixed, rateLattice):
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print "Calculating swaps"
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            if i==0:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    payment = spotRate-rf if payFixed else rf-spotRate
                    newPrice = payment/(1+spotRate)
                    newLevel.append(newPrice)
            else:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    newPrice = (q*rightLevel[j+1]+(1-q)*rightLevel[j])/(1.+spotRate)
                    if n-i >= firstPaymentTime:
                        payment = spotRate-rf if payFixed else rf-spotRate
                        newPrice += payment/(1.+spotRate)
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [10]:
numPeriods = 5
startRate = 6
upMoveReturn = 1.25
upMoveChance = .5
downMoveReturn = .9

rL = RateLattice(numPeriods, startRate, upMoveReturn, downMoveReturn)
print 'rate lattice'
print
rL.printLattice()

faceValue = 100
numPeriods = 4

zL = ZCBLattice(faceValue, upMoveChance, numPeriods, rL.lattice[:])
print 'zcb lattice'
print
zL.printLattice()
print

numPeriods = 3
strikePrice = 88
isCall = False
isAmerican = True

ozL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], zL.lattice[:])
print 'option lattice on zcb'
print
ozL.printLattice()
print

faceValue = 100
upMoveChance = .5
numPeriods = 6
couponRate = .1

cL = CBLattice(faceValue, upMoveChance, numPeriods, couponRate, rL.lattice[:])
print 'cb lattice'
print
cL.printLattice()
print

numPeriods = 4

foL = ForwardLattice(faceValue, upMoveChance, numPeriods, couponRate, rL.lattice[:], cL.lattice[:])
print 'forward lattice on cb'
foL.printLattice()
print
print 'forward price'
print faceValue*foL.lattice[0][0]/zL.lattice[0][0]
print

fuL = FutureLattice(faceValue, upMoveChance, numPeriods, couponRate, cL.lattice[:])
print 'future lattice on cb'
print
fuL.printLattice()
print

numPeriods = 6
fixedRate = .05
firstPaymentTime = 1
paysFixed = True

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, paysFixed, rL.lattice[:])
print 'swap lattice'
print
sL.printLattice()
print

numPeriods = 3
strikePrice = 0
isCall = True
isAmerican = False

osL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], sL.lattice[:])
print 'option lattice on swap'
print
osL.printLattice()
print

rate lattice

level 0
6.0
level 1
5.4, 7.5
level 2
4.86, 6.75, 9.375
level 3
4.374, 6.075, 8.438, 11.719
level 4
3.937, 5.468, 7.594, 10.547, 14.648
level 5
3.543, 4.921, 6.834, 9.492, 13.184, 18.311
Calculating prices of zcb
zcb lattice

level 0
77.218
level 1
84.434, 79.268
level 2
90.636, 87.35, 83.076
level 3
95.809, 94.273, 92.219, 89.51
level 4
100.0, 100.0, 100.0, 100.0, 100.0

Calculating options
At time 2, it's better to early exercise 0.650145069503 than hold 0.0
At time 2, it's better to early exercise 4.92365271616 than hold 0.0
At time 1, it's better to early exercise 3.56639153305 than hold 0.30841796466
At time 1, it's better to early exercise 8.73199897008 than hold 2.59246408635
At time 0, it's better to early exercise 10.7822596713 than hold 5.80112759581
option lattice on zcb

level 0
10.782
level 1
3.566, 8.732
level 2
0.0, 0.65, 4.924
level 3
0.0, 0.0, 0.0, 0.0

Calculating prices of cb
cb lattice

level 0
124.137
level 1
126.141, 115.83
level 2
126.271, 118.554, 1

In [11]:
# n = 10 period binomial model
# r0,0 = 5%, u = 1.1, d = 0.9 and q = 0.5
numPeriods = 10
startRate = 5
upMoveReturn = 1.1
downMoveReturn = .9

rL = RateLattice(numPeriods, startRate, upMoveReturn, downMoveReturn)
print 'rate lattice'
print
rL.printLattice()
print

# Q1: compute price of zcb that matures at t=10 and face value of 100
# round to 2 decimals
faceValue = 100
upMoveChance = .5
numPeriods = 10

z10L = ZCBLattice(faceValue, upMoveChance, numPeriods, rL.lattice[:])
print 'zcb lattice'
print
z10L.printLattice()

rate lattice

level 0
5.0
level 1
4.5, 5.5
level 2
4.05, 4.95, 6.05
level 3
3.645, 4.455, 5.445, 6.655
level 4
3.28, 4.01, 4.901, 5.99, 7.321
level 5
2.952, 3.609, 4.41, 5.391, 6.588, 8.053
level 6
2.657, 3.248, 3.969, 4.851, 5.93, 7.247, 8.858
level 7
2.391, 2.923, 3.572, 4.366, 5.337, 6.523, 7.972, 9.744
level 8
2.152, 2.631, 3.215, 3.93, 4.803, 5.87, 7.175, 8.769, 10.718
level 9
1.937, 2.368, 2.894, 3.537, 4.323, 5.283, 6.457, 7.892, 9.646, 11.79
level 10
1.743, 2.131, 2.604, 3.183, 3.89, 4.755, 5.812, 7.103, 8.682, 10.611, 12.969

Calculating prices of zcb
zcb lattice

level 0
61.622
level 1
67.441, 61.965
level 2
72.882, 68.07, 62.676
level 3
77.887, 73.78, 69.099, 63.838
level 4
82.422, 79.029, 75.105, 70.617, 65.556
level 5
86.475, 83.778, 80.619, 76.952, 72.741, 67.969
level 6
90.047, 88.008, 85.594, 82.755, 79.445, 75.623, 71.261
level 7
93.158, 91.723, 90.009, 87.973, 85.567, 82.745, 79.462, 75.683
level 8
95.831, 94.94, 93.868, 92.582, 91.046, 89.221, 87.063, 84.531, 81.584


In [12]:
# Q2: forward contract matures at time t = 4 using zcb
numPeriods = 4
couponRate = 0

foL = ForwardLattice(faceValue, upMoveChance, numPeriods, couponRate, rL.lattice[:], z10L.lattice[:])
z4L = ZCBLattice(faceValue, upMoveChance, numPeriods, rL.lattice[:])
print 'forward lattice on zcb'
print
foL.printLattice()
print
print 'forward price'
print faceValue*foL.lattice[0][0]/z4L.lattice[0][0]
print

Calculating forwards of bond lattice
Calculating prices of zcb
forward lattice on zcb

level 0
61.622
level 1
67.441, 61.965
level 2
72.882, 68.07, 62.676
level 3
77.887, 73.78, 69.099, 63.838
level 4
82.422, 79.029, 75.105, 70.617, 65.556

forward price
74.8848449384



In [13]:
# Q3: futures contract matures at time t = 4 using zcb
# average values at t = 4 of zcb w/ face value 100 and expiration t = 10

fuL = FutureLattice(faceValue, upMoveChance, numPeriods, couponRate, z10L.lattice[:])
print 'future lattice on zcb'
print
fuL.printLattice()
print

Calculating futures of bond lattice
future lattice on zcb

level 0
74.825
level 1
76.93, 72.719
level 2
78.896, 74.964, 70.474
level 3
80.726, 77.067, 72.861, 68.087
level 4
82.422, 79.029, 75.105, 70.617, 65.556



In [14]:
# Q4: American call option w/ expiration t = 6 and strike = 80 using zcb
numPeriods = 6
strikePrice = 80
isCall = True
isAmerican = True

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], z10L.lattice[:])
print 'option lattice on zcb'
print
oL.printLattice()

Calculating options
option lattice on zcb

level 0
2.357
level 1
3.394, 1.557
level 2
4.647, 2.445, 0.839
level 3
6.03, 3.641, 1.491, 0.289
level 4
7.423, 5.077, 2.529, 0.617, 0.0
level 5
8.769, 6.564, 3.998, 1.307, 0.0, 0.0
level 6
10.047, 8.008, 5.594, 2.755, 0.0, 0.0, 0.0


In [52]:
# Q5: forward-starting swap that w/ first payment at t = 2, final payment at t = 11 and a fixed rate of 4.5%
# payments take place in arrears. You should assume a swap notional of 1 million and assume that you receive floating and pay fixed.)
# round to nearest integer
numPeriods = 11
fixedRate = 0.045
firstPaymentTime = 2
payFixed = True

rL = RateLattice(10, startRate, upMoveReturn, downMoveReturn)

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, payFixed, rL.lattice[:])
print 'swap lattice'
print
sL.lattice
# print
# print 'price w/ notional of $1M'
# print sL.lattice[0][0]*1000000

Calculating swaps
swap lattice



[[0.03337424206216377],
 [-0.002268510618075813, 0.07235441894861974],
 [-0.03483090857491256, 0.030089721383134115, 0.10257810259845353],
 [-0.059296410171203316,
  -0.004186710573189747,
  0.058345035756388254,
  0.12822311985493168],
 [-0.07568292662395212,
  -0.030132602019935256,
  0.022286144961484552,
  0.08185770094516262,
  0.1485550360173921],
 [-0.0842317464995576,
  -0.047709663564144134,
  -0.005161873831705012,
  0.0439084288223492,
  0.09982270706419705,
  0.16262730779389342],
 [-0.0853546497681017,
  -0.05713164363006586,
  -0.02390193762731025,
  0.014913866235079332,
  0.05982580302938531,
  0.11120414938615097,
  0.16919075674915005],
 [-0.07958281639196602,
  -0.058806679186978364,
  -0.034121431140336306,
  -0.004968073528834891,
  0.029212996948396644,
  0.06894137672581507,
  0.11463960756299693,
  0.1665589805570311],
 [-0.0675211259816889,
  -0.05328061823959779,
  -0.0362290009776777,
  -0.01590110333175586,
  0.008204199764160106,
  0.03660689172249669,
  0.

In [ ]:
# Q6: swaption that matures at time t=5 and has a strike of 0
# swap is the same swap as described in the previous question with a notional of 1 million
# clarification: exercised at t=5 then the owner of the swaption will receive all cash-flows from the underlying swap from times t=6 to t=11 inclusive
# swaption is considered as a European call

numPeriods = 5
strikePrice = 0
isCall = True
isAmerican = False

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, rL.lattice[:], sL.lattice[:])
print 'option lattice on zcb'
print
oL.printLattice()
print
print 'price w/ notional of $1M'
print oL.lattice[0][0]*1000000